# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [42]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import numpy as np
import requests
import gmaps
import json

# Import API key
from api_keys import geoapify_key

In [43]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,faya,18.3851,42.4509,22.06,35,21,2.60,SA,1666108228
1,1,farsund,58.0948,6.8047,13.30,100,0,7.65,NO,1666108228
2,2,new norfolk,-42.7826,147.0587,11.72,58,12,1.34,AU,1666108230
3,3,jamestown,42.0970,-79.2353,5.77,77,100,9.77,US,1666107934
4,4,lanzhou,36.0564,103.7922,14.53,48,59,1.20,CN,1666108230


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [44]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE
map_plot = city_data_df.hvplot.points(
    "Lng", 
    "Lat", 
    geo = True, 
    size = "Humidity",
    scale = 1,
    color = "City",
    alpha = 0.5,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500
)

# Display the map
# YOUR CODE HERE
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [84]:
# Narrow down cities that fit criteria and drop any results with null values
# YOUR CODE HERE
weather_df = city_data_df.loc[(city_data_df["Max Temp"] < 24)
                             & (city_data_df["Max Temp"] > 18)
                             & (city_data_df["Wind Speed"] > 5)
                             & (city_data_df["Cloudiness"] < 4)
]

# Display sample data
# YOUR CODE HERE
weather_df


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
7,7,carnarvon,-24.8667,113.6333,20.02,76,0,7.46,AU,1666108232
9,9,saint-pierre,-21.3393,55.4781,22.82,68,0,5.14,RE,1666108236
32,32,cape town,-33.9258,18.4232,20.61,71,0,9.26,ZA,1666108250
239,239,rawson,-43.3002,-65.1023,23.74,25,2,13.52,AR,1666108102


In [85]:
# Drop any rows with null values
# YOUR CODE HERE
weather_df.dropna()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
7,7,carnarvon,-24.8667,113.6333,20.02,76,0,7.46,AU,1666108232
9,9,saint-pierre,-21.3393,55.4781,22.82,68,0,5.14,RE,1666108236
32,32,cape town,-33.9258,18.4232,20.61,71,0,9.26,ZA,1666108250
239,239,rawson,-43.3002,-65.1023,23.74,25,2,13.52,AR,1666108102


### Step 3: Create a new DataFrame called `hotel_df`.

In [86]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
# YOUR CODE HERE

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
# YOUR CODE HERE

# Display sample data
# YOUR CODE HERE


In [87]:
hotel_df = weather_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

hotel_df["Hotel Name"] = ""

hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
7,carnarvon,AU,-24.8667,113.6333,76,
9,saint-pierre,RE,-21.3393,55.4781,68,
32,cape town,ZA,-33.9258,18.4232,71,
239,rawson,AR,-43.3002,-65.1023,25,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [93]:
# Set parameters to search for a hotel
radius = 10000
params = {"categories": "accommodation",
            "apikey": geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    # YOUR CODE HERE
    lng = row["Lng"]
    lat = row["Lat"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lat},{lng}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params) 
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df.head()

Starting hotel search
carnarvon - nearest hotel: No hotel found
saint-pierre - nearest hotel: No hotel found
cape town - nearest hotel: No hotel found
rawson - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
7,carnarvon,AU,-24.8667,113.6333,76,No hotel found
9,saint-pierre,RE,-21.3393,55.4781,68,No hotel found
32,cape town,ZA,-33.9258,18.4232,71,No hotel found
239,rawson,AR,-43.3002,-65.1023,25,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [95]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE
map_plot1 = hotel_df.hvplot.points(
    "Lng", 
    "Lat", 
    geo = True, 
    size = "Humidity",
    scale = 1,
    color = "City",
    alpha = 0.5,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500
)

# Display the map
# YOUR CODE HERE
map_plot1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)